In [ ]:
import spatialdata as sd

sdata = sd.read_zarr("../data/mouse_brain_visium_hd.sdata.zarr")

print(sdata)

In [ ]:
print("Images:", list(sdata.images.keys()))
print("Tables:", list(sdata.tables.keys()))
print("Shapes:", list(sdata.shapes.keys()))
print("Labels:", list(sdata.labels.keys()))

In [ ]:
import matplotlib.pyplot as plt
import scanpy as sc
import numpy as np

adata = sdata.tables[list(sdata.tables.keys())[0]]

# --- Compute QC metrics if you haven't already ---
sc.pp.calculate_qc_metrics(adata, inplace=True)

# --- Histogram: total counts per spot (cell) ---
plt.figure(figsize=(5,4))
plt.hist(adata.obs['total_counts'], bins=100, color='steelblue')
plt.xlabel('Total counts per spot')
plt.ylabel('Number of spots')
plt.title('Distribution of counts per spot')
plt.show()

# --- Histogram: number of genes detected per spot ---
plt.figure(figsize=(5,4))
plt.hist(adata.obs['n_genes_by_counts'], bins=100, color='darkorange')
plt.xlabel('Number of detected genes per spot')
plt.ylabel('Number of spots')
plt.title('Detected genes per spot')
plt.show()

# --- Histogram: number of spots per gene ---
# Sum over axis=0 gives counts per gene
counts_per_gene = np.array((adata.X > 0).sum(axis=0)).flatten()

plt.figure(figsize=(5,4))
plt.hist(counts_per_gene, bins=100, color='seagreen')
plt.xlabel('Number of spots where gene is detected')
plt.ylabel('Number of genes')
plt.title('Detected spots per gene')
plt.show()

In [ ]:
sc.pp.filter_genes(adata, min_cells=3)
adata = adata[adata.obs['n_genes_by_counts'] > 1000, :]
adata = adata[adata.obs['total_counts'] < 100000, :]


In [ ]:
import matplotlib.pyplot as plt
import scanpy as sc
import numpy as np

adata = sdata.tables[list(sdata.tables.keys())[0]]

# --- Compute QC metrics if you haven't already ---
sc.pp.calculate_qc_metrics(adata, inplace=True)

# --- Histogram: total counts per spot (cell) ---
plt.figure(figsize=(5,4))
plt.hist(adata.obs['total_counts'], bins=100, color='steelblue')
plt.xlabel('Total counts per spot')
plt.ylabel('Number of spots')
plt.title('Distribution of counts per spot')
plt.show()

# --- Histogram: number of genes detected per spot ---
plt.figure(figsize=(5,4))
plt.hist(adata.obs['n_genes_by_counts'], bins=100, color='darkorange')
plt.xlabel('Number of detected genes per spot')
plt.ylabel('Number of spots')
plt.title('Detected genes per spot')
plt.show()

# --- Histogram: number of spots per gene ---
# Sum over axis=0 gives counts per gene
counts_per_gene = np.array((adata.X > 0).sum(axis=0)).flatten()

plt.figure(figsize=(5,4))
plt.hist(counts_per_gene, bins=100, color='seagreen')
plt.xlabel('Number of spots where gene is detected')
plt.ylabel('Number of genes')
plt.title('Detected spots per gene')
plt.show()

In [ ]:
import scanpy as sc
import numpy as np
from pathlib import Path
import pandas as pd

# Use your filtered AnnData
adata = sdata.tables[list(sdata.tables.keys())[0]]

# --- normalize + log1p ---
sc.pp.normalize_total(adata, target_sum=1e4)
# sc.pp.log1p(adata)

# (optional but recommended) focus on informative genes
sc.pp.highly_variable_genes(adata, flavor="seurat_v3", n_top_genes=2000)
adata = adata[:, adata.var.highly_variable].copy()

# PCA (20 PCs)
sc.pp.scale(adata, max_value=10)
sc.tl.pca(adata, n_comps=20)

# UMAP
sc.pp.neighbors(adata, n_pcs=20)
sc.tl.umap(adata)

# UMAP colored by library size
if "total_counts" not in adata.obs:
    sc.pp.calculate_qc_metrics(adata, inplace=True)
sc.pl.umap(adata, color="total_counts")

markers_path = Path("../data/cell_markers/normalized_cellmarkers2.csv")
markers_df = pd.read_csv(markers_path)

# Unique marker genes from file
all_markers = sorted(markers_df["marker_gene"].unique())
print(f"Markers in CSV (unique): {len(all_markers)}")

# Keep only markers that are STILL present in this adata (after HVG filtering)
adata_genes = set(adata.var_names)
gene_list = [g for g in all_markers if g in adata_genes]

print(f"Markers usable in this UMAP (present in adata): {len(gene_list)}")

# Plot UMAP colored by markers
batch_size = 4
for start in range(0, len(gene_list), batch_size):
    subset = gene_list[start:start + batch_size]
    sc.pl.umap(adata, color=subset, title=subset)

In [ ]:
# use your filtered AnnData
adata = sdata.tables[list(sdata.tables.keys())[0]]

# --- normalize + log1p ---
sc.pp.normalize_total(adata, target_sum=1e4)
# sc.pp.log1p(adata)

# (optional but recommended) focus on informative genes
sc.pp.highly_variable_genes(adata, flavor="seurat_v3", n_top_genes=2000)
adata = adata[:, adata.var.highly_variable].copy()


In [ ]:
print(adata)

In [ ]:
import json
from pathlib import Path

import numpy as np
import scanpy as sc
import squidpy as sq
from PIL import Image

# Path
root = Path("../data/visium_adult_mouse_brain")
sf_path = root / "spatial" / "scalefactors_json.json"
hires_img_path = root / "spatial" / "tissue_hires_image.png"  # change if jpg

# Load AnnData from SpatialData
adata = sdata.tables[list(sdata.tables.keys())[0]].copy()

# Preprocessing (as you had it)
sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.highly_variable_genes(adata, flavor="seurat_v3", n_top_genes=2000)
adata = adata[:, adata.var.highly_variable].copy()
sc.pp.scale(adata, max_value=10)
sc.tl.pca(adata, n_comps=20)
sc.pp.neighbors(adata, n_pcs=20)
sc.tl.umap(adata)
if "total_counts" not in adata.obs:
    sc.pp.calculate_qc_metrics(adata, inplace=True)

# Load scalefactors and hires image
with open(sf_path) as f:
    sf = json.load(f)

s_hires = float(sf["tissue_hires_scalef"])
spot_diam_full = float(sf["spot_diameter_fullres"])                # pixels at full-res
spot_diam_hires = spot_diam_full * s_hires                          # pixels at hires

# Squidpy's scatter `size` is area in points^2. Convert hires-pixel diameter to points^2.
# points ≈ pixels * (72 / dpi)
dpi = 150
size_pts2 = (spot_diam_hires * (72.0 / dpi)) ** 2

# hires image as H×W×C numpy array
img_np = np.asarray(Image.open(hires_img_path))
H, W = img_np.shape[:2]

# Validate coordinate convention
# Space Ranger stores full-res (x, y). We scale inside spatial_scatter via scale_factor=s_hires.
# Optional sanity check: if scaled coords would exceed image bounds, try swapping.
xy_full = np.asarray(adata.obsm["spatial"], dtype=float)[:, :2]
xy_hires_guess = xy_full * s_hires
needs_swap = not ((xy_hires_guess[:, 0].max() <= W) and (xy_hires_guess[:, 1].max() <= H))
if needs_swap:
    # store swapped copy in a temporary slot and use that with scale_factor
    adata.obsm["spatial_swapped"] = xy_full[:, [1, 0]]
    spatial_key = "spatial_swapped"
else:
    spatial_key = "spatial"

# Marker sets
import pandas as pd

cm_path = Path("../data/cell_markers/normalized_cellmarkers2.csv")
cm = pd.read_csv(cm_path)

# Only keep genes that exist in this Visium dataset (should already be true, but safe)
cm = cm[cm["marker_gene"].isin(adata.var_names)].copy()

# Build dict: {cell_type: [marker_gene1, marker_gene2, ...]}
markers = (
    cm.groupby("cell_type")["marker_gene"]
      .apply(lambda s: list(s.unique()))
      .to_dict()
)

# (Optional) if lists are huge, limit to first few per cell type for plotting
max_genes_per_type = 5
for ct, genes in markers.items():
    markers[ct] = genes[:max_genes_per_type]

# Recompute genes_present for the plotting loop if you still want single-gene maps:
genes_present = [g for g in {g for v in markers.values() for g in v} if g in adata.var_names]

# ---------- Plot: library size on hires image ----------
sq.pl.spatial_scatter(
    adata,
    color="total_counts",
    img=img_np,
    size=size_pts2,
    scale_factor=s_hires,      # critical: aligns full-res coords to hires pixels
    alpha=0.9,
    dpi=dpi,
    spatial_key=spatial_key,   # use swapped coords if needed
)

# ---------- Plot: each marker gene on hires image ----------
for g in genes_present:
    sq.pl.spatial_scatter(
        adata,
        color=g,
        img=img_np,
        size=size_pts2,
        scale_factor=s_hires,
        alpha=0.9,
        dpi=dpi,
        spatial_key=spatial_key,
    )

# ---------- Optional: group scores then overlay ----------
def add_group_scores(adata, markers_dict):
    for grp, genes in markers_dict.items():
        gs = [g for g in genes if g in adata.var_names]
        if not gs:
            continue
        # simple mean; replace with scanpy.tl.score_genes if you prefer a z-scored approach
        adata.obs[f"{grp}_score"] = np.asarray(adata[:, gs].X).mean(axis=1)

add_group_scores(adata, markers)

for grp in markers.keys():
    key = f"{grp}_score"
    if key in adata.obs:
        sq.pl.spatial_scatter(
            adata,
            color=key,
            img=img_np,
            size=size_pts2,
            scale_factor=s_hires,
            alpha=0.9,
            dpi=dpi,
            spatial_key=spatial_key,
        )

In [ ]:

# plot: library size
sq.pl.spatial_scatter(
    adata,
    color="total_counts",
    img=img_np,
    size=200,
    scale_factor = 10.0,
    alpha=0.9,
    dpi=150,
)